In [1]:
# import pandas
import pandas as pd

# import plotly
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
#%%  Reading PKL file into a DataFrame

ethereum_tx = pd.read_pickle('files\\ethereum_tx.pkl')
bitcoin_tx = pd.read_pickle('files\\bitcoin_tx.pkl')

# assume both have the same columns: ['chain','date','tx_count']
web3_tx = pd.concat(
    [ethereum_tx, bitcoin_tx],
    ignore_index=True
)

# Convert date to datetime and chain to categorical
web3_tx['chain'] = pd.Categorical(web3_tx['chain'])
web3_tx = web3_tx.sort_values(['chain','date']).reset_index(drop=True)

print(web3_tx.info())

In [6]:
# Create the figure
fig1 = px.line(
    web3_tx,
    x='date',
    y='tx_count',
    color='chain',
    custom_data=['chain', 'tx_count'],
    title='Blockchain Activity',
    labels={
        'date': 'Date',
        'tx_count': 'Transactions',
        'chain': 'Blockchain',
    },
    color_discrete_map={
        'bitcoin': '#F7931A',  # Bitcoin Orange
        'ethereum': '#636BFF',  # Ethereum Blue
    },
    markers=False,
    line_shape='spline',
)

# Update traces to reduce marker size
fig1.update_traces(
    line=dict(width= 1.5, smoothing=0.5),
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Transactions: %{customdata[1]:,.2f}<br>" +
                  "<extra></extra>"
)

fig1.update_layout(
    template="plotly_dark",              # inherit base dark theme 
        hovermode="x unified",
        title=dict(font=dict(size=14, color='white'), xanchor="left"),
        font=dict(family="IBM Plex Sans", color="white"),
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
        margin=dict(l=10, r=10, t=40, b=10),
        yaxis=dict(type="log", title="Price (USD)", dtick=1,),
        height=500, width=705,
)

# Show the plot
fig1.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})


# get the full HTML as a string, with Plotly.js from CDN
fig1html = fig1.to_html(include_plotlyjs="cdn", full_html=True)

# inject the Google Fonts link right after <head>
font_link = '<link href="https://fonts.googleapis.com/css2?family=IBM+Plex+Sans&display=swap" rel="stylesheet">'
fig1html = fig1html.replace("<head>", f"<head>\n    {font_link}")

# save the HTML to a file
with open("plots/html/blockchain_activity.html", "w") as f:
    f.write(fig1html)